# DAIN Colab

*DAIN Colab, v1.7.1*

Based on the [original Colab file](https://github.com/baowenbo/DAIN/issues/44) by btahir. 

Enhancements by [Styler00Dollar](https://github.com/styler00dollar), [Alpha](https://github.com/AlphaGit) and [JamesCullum](https://github.com/JamesCullum). 

[Styler00Dollar's fork](https://github.com/styler00dollar/DAIN) / [Alpha's fork](https://github.com/AlphaGit/DAIN) / [Edgars' fork](https://github.com/edgarsi/DAIN).

A simple guide:
- Upload this file to your Google Colab or use [this link](https://colab.research.google.com/github/AlphaGit/DAIN/blob/master/Colab_DAIN.ipynb).
- Create a folder inside of Google Drive named "DAIN"
- Change the configuration
- Run the steps below and repeat as needed

Stuff that should be improved:
- Alpha channel will be removed automatically and won't be added back. Anything related to alpha will be converted to black.
- Adding configuration to select speed
- Auto-resume
- Copy `start_frame` - `end_frame` audio from original input to final output


# Setup

Connect your Google Drive and get a Google GPU.

In [ ]:
#@title Connect to Google Drive
from google.colab import drive
drive.mount('/content/gdrive')
print('Google Drive connected.')

In [ ]:
#@title Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

# Install dependencies.

This next step may take somewhere between 15-20 minutes. Run this only once at startup.

Look for the "Finished installing dependencies"  message.

In [ ]:
#@title Setup everything. This takes a while. Just wait ~20 minutes in total.

# Install old pytorch to avoid faulty output
%cd /content/
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install pytorch==1.1 cudatoolkit torchvision -c pytorch -y
!conda install ipykernel -y

!pip install scipy==1.1.0
!pip install imageio
!CUDA_VISIBLE_DEVICES=0
!sudo apt-get install imagemagick imagemagick-doc
print("Finished installing dependencies.")

# Clone DAIN sources
%cd /content
!git clone -b master --depth 1 https://github.com/AlphaGit/DAIN /content/DAIN
%cd /content/DAIN
!git log -1

# Building DAIN
%cd /content/DAIN/my_package/
!./build.sh
print("Building #1 done.")

# Building DAIN PyTorch correlation package.
%cd /content/DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")

# Downloading pre-trained model
%cd /content/DAIN
!mkdir model_weights
!wget -O model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

# Configuration

In [ ]:
################# Required Configurations ############################

#@markdown # Required Configuration
#@markdown Use the values in here to configure what you'd like DAIN to do.

#@markdown ## Input file
#@markdown Path (relative to the root of your Google Drive) to the input file. For instance, if you save your `example.mkv` file in your Google Drive, inside a `videos` folder, the path would be: `videos/example.mkv`. Currenly videos and gifs are supported.
INPUT_FILEPATH = "DAIN/input.mp4" #@param{type:"string"}

#@markdown ## Output file
#@markdown Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the filetype in the destination. `.mp4` is recommended for video input, `.gif` for gif inputs.
OUTPUT_FILE_PATH = "DAIN/output.mp4" #@param{type:"string"}

################# Optional configurations ############################

#@markdown # Optional Configuration
#@markdown Parameters below can be left with their defaults, but feel free to adapt them to your needs.

#@markdown ## Target FPS
#@markdown  how many frames per second should the result have. This will determine how many intermediate images are interpolated.
TARGET_FPS = 60 #@param{type:"number"}

#@markdown ## Frame input directory
#@markdown A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
FRAME_INPUT_DIR = '/content/DAIN/input_frames' #@param{type:"string"}

#@markdown ## Frame output directory
#@markdown A path, relative to your GDrive root, where you want the generated frame.
FRAME_OUTPUT_DIR = '/content/DAIN/output_frames' #@param{type:"string"}

#@markdown ## Frame mixer
#@markdown Normally we interpolate between two adjacent frames. You can try to experiment with other mixers. Claymation mixer can give smoother outputs when animation only moves every second frame at times.
FRAME_MIXER = "normal" #@param ["normal", "claymation"] {type:"string"}

#@markdown ## Start Frame
#@markdown First frame to consider from the video when processing.
START_FRAME = 1 #@param{type:"number"}

#@markdown ## End Frame
#@markdown Last frame to consider from the video when processing. To use the whole video use `-1`.
END_FRAME = -1 #@param{type:"number"}

#@markdown ## Seamless playback
#@markdown Creates a seamless loop by using the first frame as last one as well. Set this to True this if loop is intended.
SEAMLESS = False #@param{type:"boolean"}

#@markdown ## Detect scene changes
#@markdown Detects sharp changes in video, avoiding interpolating over scene changes. Slide this all the way to 1 not use scene detection at all.
SCENE_CHANGE_THRESHOLD = 0.2 #@param {type:"slider", min:0, max:1, step:0.01}

#@markdown ## Resize hotfix
#@markdown DAIN frames are a bit "shifted / smaller" compared to original input frames. This can partly be mitigated with resizing DAIN frames to the resolution +2px and cropping the result to the original resoultion with the starting point (1,1). Without this fix, DAIN tends to make "vibrating" output and it is pretty noticible with static elements like text.
#@markdown
#@markdown This hotfix tries to make such effects less visible for a smoother video playback. I do not know what DAINAPP uses as a fix for this problem, but the original does show such behaviour with the default test images. More advanced users can change the interpolation method. The methods cv2.INTER_CUBIC and cv2.INTER_LANCZOS4 are recommended. The current default value is cv2.INTER_LANCZOS4.
RESIZE_HOTFIX = True #@param{type:"boolean"}

#@markdown ## Auto-remove PNG directory
#@markdown Auto-delete output PNG dir after ffmpeg video creation. Set this to `False` if you want to keep the PNG files.
AUTO_REMOVE = True #@param{type:"boolean"}

# Interpolate frames, create video

The speed of this depends on the amount of calculation that needs to be done. You can run the whole category with one click if you collapse it.

In [ ]:
#@title Detecting FPS of input file.
%shell yes | cp -f /content/gdrive/My\ Drive/{INPUT_FILEPATH} /content/DAIN/

import os
filename = os.path.basename(INPUT_FILEPATH)

import cv2
cap = cv2.VideoCapture(f'/content/DAIN/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)
print(f"Input file has {fps} fps")

if(fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")

In [ ]:
#@title ffmpeg extract - Generating individual frame PNGs from the source file.
%shell rm -rf '{FRAME_INPUT_DIR}'
%shell mkdir -p '{FRAME_INPUT_DIR}'

if (END_FRAME==-1):
  %shell ffmpeg -i '/content/DAIN/{filename}' -vf 'select=gte(n\,{START_FRAME}-1),setpts=PTS-STARTPTS' '{FRAME_INPUT_DIR}/%05d.png'
else:
  %shell ffmpeg -i '/content/DAIN/{filename}' -vf 'select=between(n\,{START_FRAME}-1\,{END_FRAME}-1),setpts=PTS-STARTPTS' '{FRAME_INPUT_DIR}/%05d.png'

from IPython.display import clear_output
clear_output()

png_generated_count_command_result = %shell ls '{FRAME_INPUT_DIR}' | wc -l
frame_count = int(png_generated_count_command_result.output.strip())

import shutil
if SEAMLESS:
  frame_count += 1
  first_frame = f"{FRAME_INPUT_DIR}/00001.png"
  new_last_frame = f"{FRAME_INPUT_DIR}/{str(frame_count).zfill(5)}.png"
  shutil.copyfile(first_frame, new_last_frame)

print(f"{frame_count} frame PNGs generated.")

#Checking if PNGs do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha channel detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

In [ ]:
!rm -f /content/DAIN/scene_frames.log
if SCENE_CHANGE_THRESHOLD != 1:
  scene_frames = ! \
    ffmpeg -i '{FRAME_INPUT_DIR}/%05d.png'  \
            -filter:v "showinfo,select='gt(scene,{SCENE_CHANGE_THRESHOLD})',showinfo" \
            -f null - 2>&1 | grep Parsed_showinfo_2.*n: -B 1 | grep Parsed_showinfo_0 | sed -E 's/.*n: *([0-9]+).*/\1/' \
            | tee /content/DAIN/scene_frames.log

In [ ]:
%shell pip install ipyplot 

In [ ]:
import ipyplot

def frame_image(frame):
  return f'{FRAME_INPUT_DIR}/{frame:0>5d}.png'

# Output the frame before and the frame after the first few scene changes, for manual inspection
frames = [int(i) for i in scene_frames[:100]]
for f in frames:
  images = frame_image(f), frame_image(f+1)
  ipyplot.plot_images(images, img_width=150, force_b64=True)

In [ ]:
# Interpolation
%shell mkdir -p '{FRAME_OUTPUT_DIR}'
%cd /content/DAIN

!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {frame_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}' --mixer={FRAME_MIXER} --resize_hotfix {RESIZE_HOTFIX}

In [ ]:
#@title Create output video
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'

if(AUTO_REMOVE):
  !rm -rf {FRAME_OUTPUT_DIR}/*

In [ ]:
#@title [Experimental] Create video with sound
# Only run this, if the original had sound.
%cd {FRAME_OUTPUT_DIR}
%shell ffmpeg -i '/content/DAIN/{filename}' -acodec copy output-audio.aac
%shell ffmpeg -y -r {TARGET_FPS} -f image2 -pattern_type glob -i '*.png' -i output-audio.aac -shortest '/content/gdrive/My Drive/{OUTPUT_FILE_PATH}'
if (AUTO_REMOVE):
  !rm -rf {FRAME_OUTPUT_DIR}/*
  !rm -rf output-audio.aac